# Concurrecy and Parralism in Python

## pyconil 2019
### by Guy Doulberg

# Talk Metadata 

# Who am I?


* My name is Guy Doulberg
* I work in Satellogic
* I have been developing in python in the last 3 years



# Why am I doing this talk?



* Because it is important to every python developer
* Becasue I was happy to hear this talk 2 years ago when I had the need to concurrency


# What am I going to talk about?





* What is Concucurency and Parralism
* How to implement softwares that use Concurrency and Parraslism in the python stadard library 
* How to implement Concurrency and Parraslism acrross hosts (cluster)


# Which python?


* python 3.7
* I don't know enough about python 2.7

# Theortical background

# Concurrency

Consider you have two or more tasks that you need to execute on the same time. The tasks can start, work on the same time. 



# Parralism 

Consider you have two or more tasks that run exactly on the same time (in parralel paths).

![image](http://koliber.ir/wp-content/uploads/2018/07/cvp-300x300.jpeg)

# IO bound vs CPU bound

When programing, you should consider to either run you code concurrenty or in parallel, when:






* Your program can run faster if it uses more computing units (CPUs) - **CPU bound**


* Your program can run faster if it uses more bandwidth or reading/writing to several sources. - **IO bound**

It is important to identify the nature of your program becasue each of the bounds:



* Adding more compute units to IO bounded program will not help it -  (might do worst)

* Running tasks that can use only one CPU on a CPU bounded task will not help either.


A program can also be bounded by the avaiable memory of the machine,



# Dealing with IO Bounded tasks


* Break the code that is boudned by IO and run it *Concurrectly*. (scale out)
    * In a single machine you can't utilize more than your avialable disk I/O
    * In a single machine you can't utilize more than your avialable network I/O

* Scale up your disk or network card

Most of the scenarios I was dealing with, using the Network card and Disk card concurrenctly was more than enough for me.

Let's do it in python

First I will create a usecase of I/O bounded task,

A flask server that runs the following code:

```python
from flask import Flask
app = Flask(__name__)
import time

@app.route("/")
def sleep_well():
    sleep_duration = 2
    time.sleep(sleep_duration)
    return "Hello World!"
```

I am running this flask server behind a gunicorn, in order to controll the number of requests it can handle:

```shell
gunicorn --bind 0.0.0.0:5000  -w 10 wsgi
```

# Naive implemetation will be **sequnetial** 

In [1]:
import time
import requests

host = "http://localhost:5000"
start_time = time.time()

for i in range(10):
    body = requests.get(host)
    assert body.text == "Hello World!"
    
end_time = time.time()
total = end_time - start_time

print("Total time of execution: %s" % (total))

Total time of execution: 20.104703664779663


# Using concurrent.futures.ThreadPoolExecutor


In [2]:
from concurrent.futures import ThreadPoolExecutor, as_completed 
import time
import requests

host = "http://localhost:5000"

def io_bounded_task():
    body = requests.get(host)
    return body
    
start_time = time.time()
with ThreadPoolExecutor() as e:
    futures = {e.submit(io_bounded_task) for i in range(10)}
    for future in as_completed(futures):
        assert future.result().text == "Hello World!"
    
end_time = time.time()
total = end_time - start_time

print("Total time of execution: %s" % (total))

Total time of execution: 2.036428213119507


# What happens when we reach the limit?

* Depends on what is blocking us
* If we just exceed the avaiable bandwidth (by the server/client ISP, network card) - we just wouldn't be able to run more concurrent tasks
* We could also be blocked by the server we are trying to reach 

Thefore we can throttle our requests:


In [3]:
start_time = time.time()
with ThreadPoolExecutor() as e:
    futures = {e.submit(io_bounded_task) for i in range(20)}
    for future in as_completed(futures):
        assert future.result().text == "Hello World!"
    
end_time = time.time()
total = end_time - start_time

# we are being throttled by the server so it will not be broken
#It is not healthy to count on the server for these things, we should count on our own
print("Total time of execution: %s" % (total))

Total time of execution: 4.042229175567627


In [4]:
start_time = time.time()
with ThreadPoolExecutor(max_workers=10) as e:
    futures = {e.submit(io_bounded_task) for i in range(20)}
    for future in as_completed(futures):
        future.result().text == "Hello World!"
    
end_time = time.time()
total = end_time - start_time

print("Total time of execution: %s" % (total))

Total time of execution: 4.106059551239014


# asyncio
## When dealing with IO bounded tasks we can also use the asyncio module



In [7]:
import asyncio

# declaring coroutine method
async def async_io_bounded_task():
    body = requests.get(host)
    assert body.text == "Hello World!"
    
start_time = time.time()
# creating a coroutine mehtod to run several corutines
async def main():
    await asyncio.gather(async_io_bounded_task())
    
# I am running in a jupyter notebook so I have already event loop running:
await main()

# if you don't have event loop then:
# asyncio.run(main())
end_time = time.time()
total = end_time - start_time

print("Total time of execution: %s" % (total))

Total time of execution: 2.009946823120117


# Which module to choose?

I think it is a matter of taste, 

Maybe I am old fashined, but I prefer the the Exector module becasue it is easier for me to folow the code path 

